In [1]:
import numpy as np
import pandas as pd
import arabic
import pyarabic
import helpers
from helpers import Clean_data,separate_token_with_dicrites,save_h5
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from numpy import array, argmax
import os,errno,re,sys
import pickle
from itertools import product 
from pyarabic.araby import strip_tashkeel, strip_tatweel
import h5py
import random as rn
##########################
rn.seed(123456)
np.random.seed(123456)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
'''
    * Paths
'''
all_data_path = "../../data/Almoso3a_Alshe3rya/data/All_ksaied.csv"
all_data_cleaned_path = "../../data/Almoso3a_Alshe3rya/data/All_ksaied_cleaned.csv"

'''
    * Creating Directories which will contain the Encoded data.
'''
try:
    os.makedirs("../../data/Encoded/8bits/WithoutTashkeel/Eliminated/")
    os.makedirs("../../data/Encoded/8bits/WithoutTashkeel/Full_Data/")
    os.makedirs("../../data/Encoded/8bits/WithTashkeel/Eliminated/")
    os.makedirs("../../data/Encoded/8bits/WithTashkeel/Full_Data/")
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Can't create file for checkpoints or for logs please check ")
        raise
print("Paths created")

Paths created


In [3]:
'''
    * Load and Clear
'''
all_data = pd.read_csv(all_data_path, index_col=0)
print( len(all_data) )

/usr/local/lib/python3.5/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


1831770


In [4]:
'''
    * (X,Y) form of the dataset. [uncleaned]
'''
all_data.info(memory_usage='deep')
bahr = 'Bahr'
bayt = 'Bayt_Text'
#all_data  = all_data[[bayt, bahr]]
bahr = 'البحر'
bayt = 'البيت'
all_data  = all_data[[bayt, bahr]]
all_data.columns = ['Bayt_Text','Bahr']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1831770 entries, 0 to 1835252
Data columns (total 8 columns):
العصر           object
الشاعر          object
الديوان         object
القافية         object
البحر           object
الشطر الايسر    object
الشطر الايمن    object
البيت           object
dtypes: object(8)
memory usage: 1.8 GB


In [7]:
'''
    * Cleaning Data
    # Outputs:
        $ all_data_cleaned
        % Cleaned, and Ready for filtering, encoding
'''

# Computing the maximum bayt length
MAX_LEN_BAYT = np.max((all_data.Bayt_Text.apply(pyarabic.araby.strip_tashkeel).apply(len)))


# Cleaning 
all_data_cleaned = Clean_data(data_frame=all_data,
                              max_bayt_len=MAX_LEN_BAYT,
                              verse_column_name='Bayt_Text')

In [ ]:
# clean tatwieel and tanween + shadda
all_data_cleaned['Bayt_Text'] = all_data_cleaned['Bayt_Text'].apply(pyarabic.araby.strip_tatweel)
all_data_cleaned['Bayt_Text'] = all_data_cleaned['Bayt_Text'].apply(helpers.factor_shadda_tanwin)

In [6]:
# Save cleaned data.
all_data_cleaned.to_csv(all_data_cleaned_path, encoding='utf-8')

# Load Cleaned data

In [22]:
'''
    * Load the cleaned Data
'''
all_data_cleaned = pd.read_csv(all_data_cleaned_path)
all_data_cleaned = all_data_cleaned.dropna()

# Computing the maximum bayt length
MAX_LEN_BAYT = np.max((all_data_cleaned.Bayt_Text.apply(pyarabic.araby.strip_tashkeel).apply(len)))
print(MAX_LEN_BAYT)

111


In [4]:
'''
    * Filtering the data
    # Outputs:
        1 all_filtered_data
        2 eliminated_filtered_data
'''

elminated_classic_bohor = ['الوافر', 'المنسرح','المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط']

full_bohor_classes = ['الوافر', 'المنسرح', 'المديد', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل',
                         'الرجز', 'الخفيف', 'البسيط', 'المقتضب', 'الهزج', 'المضارع', 'المتدارك']

print(elminated_classic_bohor)
print(full_bohor_classes)

all_filtered_data = all_data_cleaned.loc[all_data_cleaned['Bahr'].isin(full_bohor_classes)]
eliminated_filtered_data = all_data_cleaned.loc[all_data_cleaned['Bahr'].isin(elminated_classic_bohor)]

#all_filtered_data.head().head()
#eliminated_filtered_data.head()
#full_filtered_data.iloc[:, 1].value_counts()
#eliminated_filtered_data.iloc[:, 1].value_counts()

['الوافر', 'المنسرح', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل', 'الرجز', 'الخفيف', 'البسيط']
['الوافر', 'المنسرح', 'المديد', 'المجتث', 'المتقارب', 'الكامل', 'الطويل', 'السريع', 'الرمل', 'الرجز', 'الخفيف', 'البسيط', 'المقتضب', 'الهزج', 'المضارع', 'المتدارك']


# Let's Encode <3

In [5]:
'''
    Taha; Encoding Full Data classes as OneHot vectors. 
'''

full_data_label_encoder = LabelEncoder()
full_data_integer_encoded = full_data_label_encoder.fit_transform(list(all_filtered_data["Bahr"]))
# binary encode
full_data_onehot_encoder = OneHotEncoder(sparse=False)
full_data_integer_encoded = full_data_integer_encoded.reshape(len(full_data_integer_encoded), 1)
full_data_bohor_encoded = full_data_onehot_encoder.fit_transform(full_data_integer_encoded)
save_h5('../../data/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_Y_Meters.h5', 'Y', full_data_bohor_encoded)
save_h5('../../data/Encoded/8bits/WithTashkeel/Full_Data/full_data_Y_Meters.h5', 'Y', full_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("../../encoders_full_dat.pickle", 'wb') as pre_file:
    pickle.dump(full_data_label_encoder, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ....')


'''
    Taha; This cell is entirly doublicated.
    It Encodes the eliminated data classes.
'''
eliminated_data_label_encoder = LabelEncoder()
eliminated_data_integer_encoded = eliminated_data_label_encoder.fit_transform(list(eliminated_filtered_data["Bahr"]))
# binary encode
eliminated_data_onehot_encoder = OneHotEncoder(sparse=False)
eliminated_data_integer_encoded = eliminated_data_integer_encoded.reshape(len(eliminated_data_integer_encoded), 1)
eliminated_data_bohor_encoded = eliminated_data_onehot_encoder.fit_transform(eliminated_data_integer_encoded)
save_h5('../../data/Encoded/8bits/WithoutTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
save_h5('../../data/Encoded/8bits/WithTashkeel/Eliminated/Eliminated_data_Y_Meters.h5', 'Y', eliminated_data_bohor_encoded)
#saving with pickle: https://stackoverflow.com/questions/11218477/how-can-i-use-pickle-to-save-a-dict
print('saving preprocessors....')
with open("../../encoders_eliminated_data.pickle", 'wb') as pre_file:
    pickle.dump(eliminated_data_bohor_encoded, pre_file, protocol=pickle.HIGHEST_PROTOCOL)
print('saved ...')

saving preprocessors....
saved ....
saving preprocessors....
saved ...


In [6]:
'''
    * Encoding the full data with Tashkeel
'''


X = all_filtered_data.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, MAX_LEN_BAYT))
print("full_cleaned_data_with_tashkeel_encoded encoded ..")

X_stacked = np.stack(X,axis=0)
print("stacked!")

save_h5('../../data/Encoded/8bits/WithTashkeel/Full_Data/full_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', X_stacked) 
print("full_cleaned_data_with_tashkeel_encoded saved ..")
print(X_stacked.shape)

full_cleaned_data_with_tashkeel_encoded encoded ..
stacked!
full_cleaned_data_with_tashkeel_encoded saved ..
(1691636, 111, 8)


In [10]:
'''
    * Encoding the eleiminated data with Tashkeel
'''

X = None
X_stacked = None

X = eliminated_filtered_data.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, MAX_LEN_BAYT))
print("eliminated_cleaned_data_with_tashkeel_encoded encoded ..")

X_stacked = np.stack(X,axis=0)
save_h5('../../data/Encoded/8bits/WithTashkeel/Eliminated/eliminated_data_matrix_with_tashkeel_8bitsEncoding.h5', 'X', X_stacked)

print("full_cleaned_data_with_tashkeel_encoded saved ..")
print(X_stacked.shape)

eliminated_cleaned_data_with_tashkeel_encoded encoded ..
full_cleaned_data_with_tashkeel_encoded saved ..
(1670144, 111, 8)


In [21]:
X = None
X_stacked = None

0    خليليي لا تستعجلا أن تزوودا وأن تجمعا شملي وتن...
1    فما لبثن يومن بسابقن مغنمن ولا سرعتي يومن بساب...
2    وإن تنظراني اليوم أقض لبانتن وتستوجبا مننن علي...
3    لعمرك ما نفسن بجدن رشيدةتن تؤامرني سررن لأصرم ...
4    وإن ظهرت منه قوارص جممتن وأفرع في لومي مرارن و...
Name: Bayt_Text, dtype: object

In [23]:
# Stripping Taskeel
all_data_cleaned.Bayt_Text = all_data_cleaned.Bayt_Text.apply(pyarabic.araby.strip_tashkeel)
all_filtered_data = all_data_cleaned.loc[all_data_cleaned['Bahr'].isin(full_bohor_classes)]
eliminated_filtered_data = all_data_cleaned.loc[all_data_cleaned['Bahr'].isin(elminated_classic_bohor)]

In [26]:
'''
    * Encoding all data without Tashkeel
'''

X = None
X_stacked = None

X = all_filtered_data.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, MAX_LEN_BAYT))
print("full_filtered_data_without_tashkeel encoded ..")

X_stacked = np.stack(X, axis=0)
save_h5('../../data/Encoded/8bits/WithoutTashkeel/Full_Data/full_data_matrix_without_tashkeel_8bitsEncoding.h5', 'X', X_stacked) 
print("full_filtered_data_without_tashkeel_staked saved ..")

print(X_stacked.shape)

In [28]:
'''
    * Encoding elminated data without Tashkeel
'''

X = None
X_stacked = None

X = eliminated_filtered_data.Bayt_Text.apply(lambda x: helpers.string_with_tashkeel_vectorizer(x, MAX_LEN_BAYT))
print("eliminated_filtered_data_without_tashkeel encoded ..")

X_stacked = np.stack(X, axis=0)
save_h5('../../data/Encoded/8bits/WithoutTashkeel/Eliminated/eliminated_data_matrix_without_tashkeel_8bitsEncoding.h5', 'X', X_stacked) 
print("eliminated_filtered_data_without_tashkeel_staked saved ..")

print(X_stacked.shape)

eliminated_filtered_data_without_tashkeel encoded ..
eliminated_filtered_data_without_tashkeel_staked saved ..
(1670144, 111, 8)
